# Correlating NYT Data with Market Data

### Loading in Data

In [187]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
import glob
import string
from dateutil.relativedelta import relativedelta
import numpy as np
import yfinance as yf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.ar_model import AutoReg 

In [151]:
spy = yf.download("SPY")
demsent = pd.read_csv("./Data/DailyDemFiltered.csv")
repsent = pd.read_csv("./Data/DailyRepFiltered.csv")

demsent = demsent.set_index("date")
demsent = demsent.drop(columns = ['Unnamed: 0'])
repsent = repsent.set_index("date")
repsent = repsent.drop(columns = ['Unnamed: 0'])

xlk = yf.download("XLK")
xlv = yf.download("XLV")
xlf = yf.download("XLF")
xly = yf.download("XLY")
xli = yf.download("XLI")
xlp = yf.download("XLP")
xlu = yf.download("XLU")
xle = yf.download("XLE")
xlc = yf.download("XLC")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [153]:
listOfFunds = [spy , xlk , xlv, xlf, xly , xli, xlp, xlu, xle, xlc]
EtfNames = ["spy" , "xlk" , "xlv", "xlf", "xly" , "xli", "xlp", "xlu", "xle", "xlc"]
for etf in listOfFunds:
    etf["Log Returns"] = np.log(etf["Close"]/etf["Close"].shift(1))
spy.head()

,Open,High,Low,Close,Adj Close,Volume,Log Returns
Date,,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,26.079659,1003200,NaN
1993-02-01,43.96875,44.25000,43.96875,44.25000,26.265144,480500,0.007087
1993-02-02,44.21875,44.37500,44.12500,44.34375,26.320782,201300,0.002116
1993-02-03,44.40625,44.84375,44.37500,44.81250,26.599014,529400,0.010515
1993-02-04,44.96875,45.09375,44.46875,45.00000,26.710312,531500,0.004175


In [161]:
# get correct indicies
spy , xlk , xlv, xlf, xly , xli, xlp, xlu, xle, xlc = spy['2020-10-01':'2020-11-29'] , xlk['2020-10-01':'2020-11-29'] , xlv['2020-10-01':'2020-11-29'], xlf['2020-10-01':'2020-11-29'], xly['2020-10-01':'2020-11-29'] , xli['2020-10-01':'2020-11-29'], xlp['2020-10-01':'2020-11-29'], xlu['2020-10-01':'2020-11-29'], xle['2020-10-01':'2020-11-29'], xlc['2020-10-01':'2020-11-29']
spy.tail()

,Open,High,Low,Close,Adj Close,Volume,Log Returns
Date,,,,,,,
2020-11-20,357.500000,357.720001,355.250000,355.329987,355.329987,70417300,-0.006871
2020-11-23,357.279999,358.820007,354.869995,357.459991,357.459991,63230600,0.005977
2020-11-24,360.209991,363.809998,359.290009,363.220001,363.220001,62415900,0.015985
2020-11-25,363.130005,363.160004,361.480011,362.660004,362.660004,45330900,-0.001543
2020-11-27,363.839996,364.179993,362.579987,363.670013,363.670013,28514100,0.002781


In [229]:
def corrObjective(sentiment, logReturns, lag = 0):
    '''Returns negative absolute value of the correlation between the two timeseries with given lag.'''
    cor = sentiment.corr(logReturns.shift(lag))
    return -1 * abs(cor)

def cor(sentiment, logReturns, lag = 0):
    '''Returns the correlation between the two timeseries with given lag.'''
    cor = sentiment.corr(logReturns.shift(lag))
    return cor

In [230]:
#Get maximum correlation for each party and etf
demMax = {}
repMax = {}
for party in [repsent, demsent]:
    demParty = party is demsent
    for i, fund in enumerate(listOfFunds):
        sent, returns = party["sentiment score"], fund["Log Returns"]
        maxCor = max([[cor(sent, returns, x), abs(cor(sent, returns, x)), x] for x in range(10)], key = lambda x: x[1])
        etfName = EtfNames[i]
        if demParty:
            demMax[etfName] = {"correlation": maxCor[0], "shift": maxCor[2]}
        else:
            repMax[etfName] = {"correlation": maxCor[0], "shift": maxCor[2]}

In [238]:
repCorDf = pd.DataFrame(data=repMax).T
demCorDf = pd.DataFrame(data=demMax).T

AutoReg(spy["Log Returns"], exog=repsent["sentiment score"], lags=8).fit().bic
repCorDf


,correlation,shift
spy,-0.350207,3.0
xlk,-0.344887,3.0
xlv,-0.351322,5.0
xlf,0.170751,1.0
xly,-0.294151,3.0
xli,-0.286713,3.0
xlp,-0.412372,5.0
xlu,0.299948,8.0
xle,-0.231486,3.0
xlc,-0.320619,5.0
